In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Data Preprocessing
Data preprocessing transforms raw data into a clean and usable format by handling missing values, outliers, and ensuring consistent data scales through normalization or standardization. It also includes feature extraction and selection to enhance dataset quality. This step is essential for efficient and accurate data analysis or machine learning model performance.

In [ ]:
# importing all the necessary libraries
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.sparse import hstack
import numpy as np

In [ ]:
# Reading and displaying the first 10 rows of the CSV file
df=pd.read_csv('/content/drive/MyDrive/fake_job/fake_job_postings.csv')
df.head(10)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
5,6,Accounting Clerk,"US, MD,",NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
7,8,Lead Guest Service Specialist,"US, CA, San Francisco",NaN,NaN,Airenvy’s mission is to provide lucrative yet ...,Who is Airenvy?Hey there! We are seasoned entr...,"Experience with CRM software, live chat, and p...",Competitive Pay. You'll be able to eat steak e...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
8,9,HP BSM SME,"US, FL, Pensacola",NaN,NaN,Solutions3 is a woman-owned small business who...,Implementation/Configuration/Testing/Training ...,MUST BE A US CITIZEN.An active TS/SCI clearanc...,NaN,0,1,1,Full-time,Associate,NaN,Information Technology and Services,NaN,0
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Customer Service Associate will be based i...,Minimum Requirements:Minimum of 6 months custo...,NaN,0,1,0,Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0


In [ ]:
# Fill missing values in text columns with empty strings
text_columns = ['description', 'requirements', 'company_profile', 'title', 'location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
df[text_columns] = df[text_columns].fillna('')

In [ ]:
# Combine text columns into a single column for vectorization
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

In [ ]:
# Simplified text preprocessing
df['combined_text'] = df['combined_text'].str.lower()
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x))
df['combined_text'] = df['combined_text'].str.replace('\n', ' ')
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
df['combined_text'] = df['combined_text'].str.strip()
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(' +', ' ', x))
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)  # Limit to the top 5000 words
tokenizer.fit_on_texts(df['combined_text'])
X_text = tokenizer.texts_to_sequences(df['combined_text'])

# Pad sequences to the same length
max_length = 100
X_text_padded = pad_sequences(X_text, maxlen=max_length, padding='post')

In [ ]:
# List of non-text columns to be used as features
non_text_columns = ['telecommuting', 'has_company_logo', 'has_questions', 'location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']

In [ ]:
# Fill missing values in non-text columns with a placeholder
df[non_text_columns] = df[non_text_columns].fillna('missing')

In [ ]:
# OneHotEncode the categorical non-text columns
categorical_columns = ['location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
encoder = OneHotEncoder()
X_non_text_encoded = encoder.fit_transform(df[categorical_columns])

In [ ]:
# Scale non-categorical non-text columns
scaler = StandardScaler()
X_non_text_scaled = scaler.fit_transform(df[['telecommuting', 'has_company_logo', 'has_questions']])

In [ ]:
# Combine non-categorical and categorical features
from scipy.sparse import hstack
X_non_text = hstack([X_non_text_encoded, X_non_text_scaled]).toarray()

In [ ]:
# Combine the text and non-text features
X_combined = np.hstack((X_text_padded, X_non_text))

##Model Building
Model building with a BERT-based model involves tokenizing text data using the BERT tokenizer, selecting a pre-trained BERT model like `bert-base-uncased`, and adding a task-specific layer. The model is fine-tuned on the dataset with a smaller learning rate and fewer epochs. Training is performed using appropriate loss functions and optimizers, and the model's performance is evaluated using metrics like accuracy or F1 score. Libraries such as Hugging Face's Transformers facilitate this process, allowing for efficient implementation and fine-tuning.

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Split the data into training, testing, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X_combined, df['fraudulent'], test_size=0.3, random_state=42
)
X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.33, random_state=42
)  # 0.33 * 0.3 = 0.10

In [ ]:
# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input data
def tokenize_data(texts):
    # Assuming the first column of your arrays contains the text data
    return tokenizer([str(row[0]) for row in texts], padding=True, truncation=True, return_tensors="pt")

train_encodings = tokenize_data(X_train.tolist())
val_encodings = tokenize_data(X_val.tolist())
test_encodings = tokenize_data(X_test.tolist())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Convert the labels to tensors with Float type
train_labels = torch.tensor(y_train.values, dtype=torch.float32)
val_labels = torch.tensor(y_val.values, dtype=torch.float32)
test_labels = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
# Define a Dataset class
class FraudDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FraudDataset(train_encodings, train_labels)
val_dataset = FraudDataset(val_encodings, val_labels)
test_dataset = FraudDataset(test_encodings, test_labels)

In [ ]:
# Load pre-trained BERT model with regression head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.027000,0.055569
2,0.000800,0.055124


Epoch,Training Loss,Validation Loss
1,0.027000,0.055569
2,0.000800,0.055124
3,0.024300,0.053053


TrainOutput(global_step=4695, training_loss=0.04768488907986206, metrics={'train_runtime': 11317.8445, 'train_samples_per_second': 3.318, 'train_steps_per_second': 0.415, 'total_flos': 135067258654776.0, 'train_loss': 0.04768488907986206, 'epoch': 3.0})

##Model Evaluation
Model evaluation involves assessing a trained model's performance using various metrics to determine its effectiveness and generalization ability. Common metrics for classification tasks include accuracy, precision, recall, F1 score, and AUC-ROC, while regression tasks often use mean squared error (MSE), mean absolute error (MAE), and R-squared. The evaluation process typically includes splitting the data into training and testing sets, training the model on the training set, and evaluating it on the test set to ensure the model performs well on unseen data. Cross-validation can also be used for more robust evaluation.

In [ ]:
# Evaluate the model
eval_result = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Loss (MSE): {eval_result['eval_loss']}")

Test Loss (MSE): 0.045591358095407486


In [ ]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.flatten()

In [ ]:
from sklearn.metrics import mean_absolute_error
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 0.08050347002979494


##Conclusion
The model exhibits a Mean Squared Error (MSE) of 0.0456 and a Mean Absolute Error (MAE) of 0.0805 on the test data, indicating that it performs well with a relatively low level of prediction error. These metrics suggest that the model is effective at predicting values close to the actual targets, demonstrating good accuracy for the regression task.

#Comparison
Let's compare the three models based on their performance metrics:

### Logistic Regression
- **Accuracy**: 0.9757
- **Precision (Class 0)**: 0.98
- **Recall (Class 0)**: 1.00
- **F1-Score (Class 0)**: 0.99
- **Precision (Class 1)**: 0.94
- **Recall (Class 1)**: 0.61
- **F1-Score (Class 1)**: 0.74
- **Macro Avg Precision**: 0.96
- **Macro Avg Recall**: 0.80
- **Macro Avg F1-Score**: 0.86
- **Weighted Avg Precision**: 0.97
- **Weighted Avg Recall**: 0.98
- **Weighted Avg F1-Score**: 0.97

### LSTM Model
- **Test Loss (MSE)**: 0.0411
- **Mean Absolute Error (MAE)**: 0.0799

### BERT-Based Model
- **Test Loss (MSE)**: 0.0456
- **Mean Absolute Error (MAE)**: 0.0805

### Comparison and Recommendation:

1. **Logistic Regression**:
   - **Pros**:
     - Very high overall accuracy (97.57%).
     - Excellent performance on the majority class (Class 0).
   - **Cons**:
     - Lower recall for the minority class (Class 1), which may indicate that it misses a significant portion of the positive cases.
   - **Best for**: Situations where the overall accuracy is crucial, but the model might not be as effective in identifying the minority class.

2. **LSTM Model**:
   - **Pros**:
     - Low Mean Squared Error (MSE) and Mean Absolute Error (MAE), indicating good predictive performance.
   - **Cons**:
     - LSTM models are typically more complex and resource-intensive to train and deploy.
   - **Best for**: Time series data or sequential data where capturing the temporal dependencies is crucial.

3. **BERT-Based Model**:
   - **Pros**:
     - Comparable performance to LSTM with slightly higher MSE and MAE.
   - **Cons**:
     - Similar to LSTM, BERT models are computationally expensive and complex.
   - **Best for**: Text data or situations where contextual understanding is important.

### Recommendation:

- If we prioritize overall accuracy and have a significant class imbalance, **Logistic Regression** might be the best choice despite its lower performance on the minority class.
- If the data involves sequences or time-series data, and we need a more nuanced model to capture these dependencies, go for the **LSTM Model**.
- For text data or when we need to leverage contextual information, the **BERT-Based Model** is suitable.

Considering the metrics, the **Logistic Regression** model shows the best overall performance in terms of accuracy and weighted averages. However, for more complex or specific tasks, either the LSTM or BERT model could be more appropriate.
